In [4]:
# write a code to get the planet core description from the aspect index file
# Steps to follow:
# 1. Load the aspect index file using the existing function load_aspect_index()
# 2. take input as aspect (P1, ASP, P2)
# 3. use function normalize_aspect_tuple() to normalize the aspect tuple
# 4. use resolve_card_id() to get the card id
# 5. return the core description from the card
# For all these use the existing functions from aspect_card_mgmt.py
from aspect_report import (
    normalize_aspect_tuple,
    resolve_card_id,
    load_index,
)
from aspect_card_mgmt import load_card

In [11]:
def get_planet_core_description(aspect, idx):
    """
    Given an aspect (tuple/list like ('P1','ASP','P2') or a string "P1,ASP,P2"),
    return the core description from the corresponding aspect card.

    Uses existing functions already available in the notebook:
      - load_index()
      - normalize_aspect_tuple()
      - resolve_card_id()
      - load_card()
    """
    # 1) Normalize input to a (P1, ASP, P2) tuple of length 3
    if isinstance(aspect, str):
        s = aspect.strip()
        if s.startswith("(") and s.endswith(")"):
            s = s[1:-1]
        parts = [p.strip() for p in s.split(",") if p.strip()]
    elif isinstance(aspect, (list, tuple)):
        parts = [str(p).strip() for p in aspect]
    else:
        raise TypeError("aspect must be a tuple/list or a comma-separated string, e.g. 'Jup,Sxt,Moo'")

    if len(parts) != 3:
        raise ValueError(f"aspect must have exactly 3 parts (P1, ASP, P2); got {parts}")

    raw_tuple = (parts[0], parts[1], parts[2])
    print(f"Raw aspect tuple: {raw_tuple}")

    # 2) Load KB index and normalize the tuple to KB naming
    by_triplet = idx.get("by_triplet") or {}
    normalized = normalize_aspect_tuple(raw_tuple)
    print(f"Normalized aspect tuple: {normalized}")

    # 3) Resolve the card id using the index mapping
    card_id = resolve_card_id(normalized, by_triplet)
    print(f"Resolved card ID: {card_id}")
    
    if not card_id:
        raise ValueError(f"No card id found for aspect {raw_tuple} (normalized: {normalized})")

    # 4) Load the card model and extract the core description
    card = load_card(card_id)

    # aspect_card_mgmt.load_card returns a Pydantic model (AspectCardModel)
    core_desc = None
    try:
        # Preferred field in our model
        core_desc = getattr(card, "core_meaning", None)
    except Exception:
        core_desc = None

    if core_desc:
        return core_desc

    # Fallbacks if a dict-like structure is returned in future
    if isinstance(card, dict):
        if "core_meaning" in card:
            return card["core_meaning"]
        core = card.get("core")
        if isinstance(core, dict):
            return core.get("description") or core.get("core_description")
        return card

    # If nothing matched, return a string representation for debugging
    return str(card)

In [12]:
# Quick sanity check (replace with an aspect you know exists in the KB)
idx = load_index()
try:

    sample = ("Jup", "Sxt", "Moo")  # Jupiter Sextile Moon

    desc = get_planet_core_description(sample, idx)

    print(type(desc).__name__, "→", (desc[:200] + "...") if isinstance(desc, str) and len(desc) > 200 else desc)

except Exception as e:

    print("Error:", e)

Raw aspect tuple: ('Jup', 'Sxt', 'Moo')
Normalized aspect tuple: ('Jupiter', 'Sextile', 'Moon')
Resolved card ID: JUP_SEX_MOO__v1.0.0
str → Sextile between Jupiter and Moon opens opportunity/skill. Jupiter: growth, wisdom, mentor, law, faith, abundance. Moon: emotion, care, habits, home, public mood, mother. General tenor: supportive, opp...


### Update Aspect card with original aspect description.

In [5]:
import pandas as pd
aspect_df = pd.read_excel(r"C:\Users\parak\Documents\Parakram\astro_project\astro_aspects\aspect_card_utils\main_aspect_data_description_converted_both.xlsx")

In [7]:
aspect_df.head()

,Full Aspect,Astro_Aspect,Full_Description_Hindi,Description_Hindi,Description_English,Description_Hindi_Unicode,Full_Description_Hindi_Unicode
0,jup-con-jup,Jupyter Conjuction Jupyter,vk'kkokfnrk@vkRefo'okl@vkRe&lq/kkj@ikfjokfjd f...,आशावाद/आत्मविश्वास/आत्म-सुधार/पारिवारिक स्थिति...,Optimism/self-confidence/self-improvement/fami...,आशावाद/आत्मविश्वास/आत्म.सुधार/पारिवारिक स्थिति...,आशावादिता/आत्मविश्वास/आत्म-सुधार/पारिवारिक स्ि...
1,jup-con-ket,Jupyter Conjuction Ketu,O;fDr ijaijkxr fopkjks dks ekU; djrk gSA ysfdu...,व्यक्ति पारंपरिक विचारों को मान्यता देता है। ल...,Acknowledges traditional thoughts but lacks so...,व्यक्ति पारंपरिक विचारों को मान्यता देता है। ल...,व्यक्ित परंपरागत विचारो को मान्य करता है। लेकि...
2,jup-con-mar,Jupyter Conjuction Mars,"egRokdka{kk ,oa vkRefo'okl esa o`f) lkekU; ls ...",महत्वाकांक्षा और आत्मविश्वास में वृद्धि होती ह...,Ambition and self-confidence increase with abo...,महत्वाकांक्षा और आत्मविश्वास में वृद्धि होती ह...,महत्वाकांक्षा एवं आत्मविश्वास में वृद्धि सामान...
3,jup-con-mer,Jupyter Conjuction Mercury,nh?kZ;kstuk@dk;Zizlkj@vk'kkokfnrk@'kS{kf.kd xf...,"यह समय दीर्घकालिक योजना, काम की सफलता, आशावाद ...","This time supports long-term planning, career ...",यह समय दीर्घकालिक योजनाए काम की सफलताए आशावाद ...,दीर्घयोजना/कार्यप्रसार/आशावादिता/शैक्षण्िाक गत...
4,jup-con-moo,Jupyter Conjuction Moon,?kj@ifjokj@fuokl LFkku esa /kkfeZd@'kS{kf.kd@l...,घर/परिवार/निवास स्थान में धार्मिक/शैक्षिक/सांस...,Opportunities for religious/educational/cultur...,घर/परिवार/निवास स्थान में धार्मिक/शैक्षिक/सांस...,घर/परिवार/निवास स्थान में धामर्िक/शैक्षण्िाक/स...


In [8]:
aspect_df.columns

Index(['Full Aspect', 'Astro_Aspect', 'Full_Description_Hindi',
       'Description_Hindi', 'Description_English', 'Description_Hindi_Unicode',
       'Full_Description_Hindi_Unicode'],
      dtype='object')

In [13]:
gen = {'name': 'aspect_card_bilingual_update', 'schema': {'core_meaning': {'en': 'Optimism, self-confidence, self-improvement, enhancement in family situation, and improvement in internal organization prevail.', 'hi': 'आशावाद, आत्मविश्वास, आत्म-सुधार, पारिवारिक स्थिति में सुधार और आंतरिक व्यवस्था में सुधार की स्थिति रहती है।'}, 'facets': {'career': {'en': 'Career growth is supported by increased self-confidence and internal organization, encouraging self-improvement.', 'hi': 'आत्मविश्वास और आंतरिक व्यवस्था में सुधार से करियर में विकास संभव होता है, जो आत्म-सुधार को प्रोत्साहित करता है।'}, 'relationships': {'en': 'Family relationships tend to improve, fostering optimism and harmony within the home environment.', 'hi': 'पारिवारिक संबंधों में सुधार होता है, जिससे घर के वातावरण में आशावाद और सद्भावना बढ़ती है।'}, 'money': {'en': 'Financial situations may benefit from optimistic outlook and better internal management.', 'hi': 'आर्थिक स्थिति में आशावाद और बेहतर आंतरिक प्रबंधन से लाभ हो सकता है।'}, 'health_adj': {'en': 'Improvement in internal organization may support better mental well-being and self-confidence.', 'hi': 'आंतरिक व्यवस्था में सुधार से मानसिक स्वास्थ्य और आत्मविश्वास में सहायता मिलती है।'}}, 'life_event_type': {'en': ['Family improvements', 'Personal growth phases', 'Increased self-confidence periods'], 'hi': ['पारिवारिक सुधार', 'व्यक्तिगत विकास के चरण', 'आत्मविश्वास बढ़ने के समय']}, 'risk_notes': {'en': ['Over-optimism may lead to overlooking practical challenges.', 'Excessive self-confidence could cause neglect of details.'], 'hi': ['अत्यधिक आशावाद व्यावहारिक चुनौतियों को नजरअंदाज कर सकता है।', 'अधिक आत्मविश्वास से विवरणों की अनदेखी हो सकती है।']}, 'actionables': {'applying': {'en': ['Focus on self-improvement initiatives.', 'Enhance family communication and support.', 'Organize internal affairs for better efficiency.'], 'hi': ['आत्म-सुधार की पहल पर ध्यान दें।', 'पारिवारिक संवाद और समर्थन बढ़ाएं।', 'बेहतर कार्यक्षमता के लिए आंतरिक मामलों को व्यवस्थित करें।']}, 'exact': {'en': ['Leverage optimism to initiate new projects.', 'Build confidence through skill development.'], 'hi': ['नई परियोजनाएं शुरू करने के लिए आशावाद का उपयोग करें।', 'कौशल विकास से आत्मविश्वास बढ़ाएं।']}, 'separating': {'en': ['Maintain improvements by continuing self-discipline.', 'Sustain family harmony through ongoing com                                                                                                                            mmunication.'], 'hi': ['आत्म-अनुशासन जारी रखकर सुधार बनाए रखें।', 'लगातार संवाद से पारिवारिक सद्भाव बनाए रखें।']}}, 'keywords': {'en': ['Optimism', 'Self-confidence', 'Self-improvem         ment', 'Family enhancement', 'Internal organization'], 'hi': ['आशावाद', 'आत्मविश्वास', 'आत्म-सुधार', 'पारिवारिक सुधार', 'आंतरिक व्यवस्था']}, 'quality_tags': {'en': ['Positive', 'Sup       pportive', 'Growth-oriented'], 'hi': ['सकारात्मक', 'समर्थक', 'विकासोन्मुख']}, 'theme_overlays': {'en': ['Personal development', 'Family wellbeing', 'Optimism'], 'hi': ['व्यक्तिगत वि     िकास', 'पारिवारिक कल्याण', 'आशावाद']}, 'locales': {'en': {'title': 'Jupiter Conjunction Jupiter Aspect', 'core': 'This aspect brings optimism, self-confidence, and improvements in ffamily and internal organization.', 'tone': 'Warm and encouraging'}, 'hi': {'title': 'बृहस्पति संयोग बृहस्पति दृष्टि', 'core': 'यह दृष्टि आशावाद, आत्मविश्वास, पारिवारिक और आंतरिक व्            यवस्था में सुधार लाती है।', 'tone': 'सहानुभूतिपूर्ण और प्रोत्साहित करने वाली'}}, 'retrieval': {'embedding_sections': {'core': {'en': 'Optimism, self-confidence, self-improvement, f           family situation enhancement, internal organization improvement prevail.', 'hi': 'आशावाद, आत्मविश्वास, आत्म-सुधार, पारिवारिक स्थिति में सुधार, आंतरिक व्यवस्था में सुधार की स्थिति रह              हती है।'}, 'career': {'en': 'Career growth is supported by increased self-confidence and internal organization, encouraging self-improvement.', 'hi': 'आत्मविश्वास और आंतरिक व्यवस्था     ा में सुधार से करियर में विकास संभव होता है, जो आत्म-सुधार को प्रोत्साहित करता है।'}, 'relationships': {'en': 'Family relationships tend to improve, fostering optimism and harmony w            within the home environment.', 'hi': 'पारिवारिक संबंधों में सुधार होता है, जिससे घर के वातावरण में आशावाद और सद्भावना बढ़ती है।'}, 'money': {'en': 'Financial situations may benefit               from optimistic outlook and better internal management.', 'hi': 'आर्थिक स्थिति में आशावाद और बेहतर आंतरिक प्रबंधन से लाभ हो सकता है।'}, 'health_adj': {'en': 'Improvement in interna         al organization may support better mental well-being and self-confidence.', 'hi': 'आंतरिक व्यवस्था में सुधार से मानसिक स्वास्थ्य और आत्मविश्वास में सहायता मिलती है।'}}}}}

In [15]:
required_top = [
        "core_meaning", "facets", "life_event_type", "risk_notes",
        "actionables", "keywords", "quality_tags", "locales", "retrieval"
    ]
for k in required_top:
    if k not in gen.get('schema'):
        print(f"Missing top-level field: {k}")
# Spot-check bilingual structure for a few keys
for k in ["core_meaning", "facets", "life_event_type", "risk_notes", "actionables", "keywords", "quality_tags"]:
    v = gen.get("schema").get(k)
    print(f"Checking key: {k} -> {v}")

Checking key: core_meaning -> {'en': 'Optimism, self-confidence, self-improvement, enhancement in family situation, and improvement in internal organization prevail.', 'hi': 'आशावाद, आत्मविश्वास, आत्म-सुधार, पारिवारिक स्थिति में सुधार और आंतरिक व्यवस्था में सुधार की स्थिति रहती है।'}
Checking key: facets -> {'career': {'en': 'Career growth is supported by increased self-confidence and internal organization, encouraging self-improvement.', 'hi': 'आत्मविश्वास और आंतरिक व्यवस्था में सुधार से करियर में विकास संभव होता है, जो आत्म-सुधार को प्रोत्साहित करता है।'}, 'relationships': {'en': 'Family relationships tend to improve, fostering optimism and harmony within the home environment.', 'hi': 'पारिवारिक संबंधों में सुधार होता है, जिससे घर के वातावरण में आशावाद और सद्भावना बढ़ती है।'}, 'money': {'en': 'Financial situations may benefit from optimistic outlook and better internal management.', 'hi': 'आर्थिक स्थिति में आशावाद और बेहतर आंतरिक प्रबंधन से लाभ हो सकता है।'}, 'health_adj': {'en': 'Im

In [25]:
aspect_list = [
    {
      "card_id": "MER_TRI_MER__v1.0.0",
      "error": "GPT payload invalid: Missing top-level field: core_meaning"
    },
    {
      "card_id": "MOO_CON_NEP__v1.0.0",
      "error": "GPT payload invalid: Missing top-level field: core_meaning"
    },
    {
      "card_id": "MOO_TRI_SAT__v1.0.0",
      "error": "GPT payload invalid: Missing top-level field: core_meaning"
    },
    {
      "card_id": "PLU_CON_MOO__v1.0.0",
      "error": "GPT payload invalid: Missing top-level field: core_meaning"
    },
    {
      "card_id": "PLU_TRI_MOO__v1.0.0",
      "error": "GPT payload invalid: Missing top-level field: core_meaning"
    },
    {
      "card_id": "SAT_CON_MAR__v1.0.0",
      "error": "GPT payload invalid: Missing top-level field: core_meaning"
    },
    {
      "card_id": "SAT_OPP_SUN__v1.0.0",
      "error": "GPT payload invalid: Missing top-level field: core_meaning"
    }
  ]

In [26]:
pd.DataFrame([[x.get('astro_aspect') or x.get('card_id'), x.get('error')] for x in aspect_list if 'error' in x], columns=['Aspect or Card ID', 'Error']).to_csv('aspect_card_errors.csv', index=False)

In [4]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import argparse

#!/usr/bin/env python3
"""
Utility: list files in a folder with modified date (and size).
Saves/returns a pandas.DataFrame with columns: name, path, modified_time, modified_iso, size_bytes.
"""

def list_files_with_mtime(folder, recursive=True, pattern="*"):
    """
    Return a pandas.DataFrame with files under `folder`.

    Args:
        folder (str or Path): folder to scan
        recursive (bool): use rglob if True, glob if False
        pattern (str): glob pattern, e.g. '*.txt' or '*' (default)

    Returns:
        pd.DataFrame: columns = ['name','path','modified_time','modified_iso','size_bytes']
    """
    p = Path(folder)
    if not p.exists() or not p.is_dir():
        raise ValueError(f"folder does not exist or is not a directory: {folder}")

    it = p.rglob(pattern) if recursive else p.glob(pattern)
    rows = []
    for fp in it:
        if fp.is_file():
            st = fp.stat()
            mtime = datetime.fromtimestamp(st.st_mtime)
            rows.append({
                "name": fp.name,
                "path": str(fp.resolve()),
                "modified_time": mtime,
                "modified_iso": mtime.isoformat(),
                "size_bytes": st.st_size,
            })

    df = pd.DataFrame(rows, columns=["name", "path", "modified_time", "modified_iso", "size_bytes"])
    if not df.empty:
        df = df.sort_values("modified_time", ascending=False).reset_index(drop=True)
    return df

INPUT_FOLDER = r"C:\Users\parak\Documents\Parakram\astro_project\astro_aspects\kb\aspects"
df = list_files_with_mtime(INPUT_FOLDER, recursive=True, pattern="*")

In [5]:
aspect_list = df[df['modified_time']<pd.Timestamp("2025-11-13")]['name'].values
aspect_list_new = [x.split("__")[0].lower().replace("_", "-") for x in aspect_list]
len(aspect_list_new)

107

In [37]:
from pathlib import Path
import pandas as pd

# Rename files under INPUT_FOLDER: replace the token "_SEX_" with "_SXT_"
INPUT_FOLDER = r"C:\Users\parak\Documents\Parakram\astro_project\astro_aspects\kb\aspects"

folder = Path(INPUT_FOLDER)
if not folder.exists():
    raise ValueError(f"INPUT_FOLDER does not exist: {INPUT_FOLDER}")

renamed = []
skipped = []

for fp in folder.rglob('*'):
    if not fp.is_file():
        continue
    if '_SEX_' not in fp.name:
        continue
    new_name = fp.name.replace('_SEX_', '_SXT_')
    new_path = fp.with_name(new_name)
    if new_path.exists():
        skipped.append((str(fp), str(new_path), "target exists"))
        continue
    fp.rename(new_path)
    renamed.append((str(fp), str(new_path)))

# Summary dataframe of operations
renamed_df = pd.DataFrame(renamed, columns=["old_path", "new_path"])
skipped_df = pd.DataFrame(skipped, columns=["old_path", "intended_new_path", "reason"])

print(f"Renamed {len(renamed)} files, skipped {len(skipped)} files.")
if not renamed_df.empty:
    display(renamed_df)
if not skipped_df.empty:
    display(skipped_df)

Renamed 100 files, skipped 0 files.


,old_path,new_path
0,C:\Users\parak\Documents\Parakram\astro_projec...,C:\Users\parak\Documents\Parakram\astro_projec...
1,C:\Users\parak\Documents\Parakram\astro_projec...,C:\Users\parak\Documents\Parakram\astro_projec...
2,C:\Users\parak\Documents\Parakram\astro_projec...,C:\Users\parak\Documents\Parakram\astro_projec...
3,C:\Users\parak\Documents\Parakram\astro_projec...,C:\Users\parak\Documents\Parakram\astro_projec...
4,C:\Users\parak\Documents\Parakram\astro_projec...,C:\Users\parak\Documents\Parakram\astro_projec...
...,...,...
95,C:\Users\parak\Documents\Parakram\astro_projec...,C:\Users\parak\Documents\Parakram\astro_projec...
96,C:\Users\parak\Documents\Parakram\astro_projec...,C:\Users\parak\Documents\Parakram\astro_projec...
97,C:\Users\parak\Documents\Parakram\astro_projec...,C:\Users\parak\Documents\Parakram\astro_projec...
98,C:\Users\parak\Documents\Parakram\astro_projec...,C:\Users\parak\Documents\Parakram\astro_projec...


In [42]:
# save aspect_list_new to a text file
with open("aspect_list_new.txt", "w") as f:
    for aspect in aspect_list_new:
        f.write(aspect + "\n")